# 12주
블록체인에서 주문을 하고, 그 주문내역을 로컬 파일에 작성하세요.

블록체인으로 주문이 전송되면 이벤트가 발생하도록 프로그램하고, 주문내역은 로컬파일에 저장됩니다.

* 강의자료 OrderEvent.sol을 참조해서 구현하세요.

	* order(상품항목, 주문개수, 단가, 배송지) 함수를 작성. '개수 x 단가'로 주문금액을 계산하고 이 금액만큼 입금되지 않으면 예외발생.

	* getBalance() 함수를 작성. 최초 프로그램 생성자 owner만 읽을 수 있게 modfier isOwner 작성.

(1) REMIX에서 주문하고, 이벤트 발생하는지 보이도록 콘솔 출력 붙여넣으세요.

(2) 파일로 컴파일 하세요.

(3) ganache에서 배포하세요.

(4) 주문을 3건만 하고, 이벤트를 통해 다음과 같이 출력하세요.

항목은 차례대로 주소, 상품항목번호, 주문개수, 단가, 배송지.

```

전송자주소, 1111, 3, 100, 20 2-gil Hongji-dong Jongro-gu Seoul

전송자주소, 1111, 5, 100, 20 2-gil Hongji-dong Jongro-gu Seoul

전송자주소, 1111, 20, 100, 20 2-gil Hongji-dong Jongro-gu Seoul

```

(5) 파일에 저장된 파일내역을 출력하세요 (주피터 노트북에서 !type src/OrderEvent.txt)

## 함수작성

In [7]:
%%writefile src/OrderEvent1.sol
//SPDX-License-Identifier: GPL-3.0-or-later
pragma solidity ^0.6.4;

contract OrderEvent1 {
    
    address payable owner;
    constructor() public payable {
        owner = msg.sender;
    }
    
    event OrderLog(string);
    event OrderLog(bytes2 _itemId, uint _value);
    event OrderLog(uint timestamp);
    event OrderLog(address indexed _from, bytes2 _itemId, uint _quantity, uint unitPrice, string addr);

    function order(bytes2 _itemId, uint quantity,uint unitPrice, string memory addr) public payable {
        uint256 orderTime = block.timestamp;
        uint256 orderAmount = quantity * unitPrice;
        
        require(msg.value == orderAmount);
        emit OrderLog("Ordered");
        emit OrderLog(orderTime);
        emit OrderLog(msg.sender, _itemId, quantity, unitPrice, addr);
    }
    function getBalance() public isOwner view returns(uint) {
        return owner.balance;
    }
    modifier isOwner {
        if(msg.sender != owner)
        {
            revert();
        }
        _;
    }
}

Overwriting src/OrderEvent1.sol


## REMIX 배포 결과

[vm]from: 0x5B3...eddC4to: Order.(constructor)value: 10000000 weidata: 0x608...40033logs: 0hash: 0x9d8...309b3

status	true Transaction mined and execution succeed

transaction hash	0x9d885aa2f58cfbcfca68a5a9fcc5c1ef164b3817f682e2b915d02de842d309b3

from	0x5B38Da6a701c568545dCfcB03FcB875f56beddC4

to	Order.(constructor)

gas	337161 gas

transaction cost	293183 gas 

execution cost	293183 gas 

input	0x608...40033

decoded input	{}

decoded output	 - 

logs	[]

val	10000000 wei

## 파일 컴파일

In [11]:
!solc.exe --optimize --combined-json abi,bin src/OrderEvent1.sol > src/OrderEvent1.json

## 배포

In [14]:
%%writefile src/OrderEvent1Deploy.js
var Web3 = require('web3');
var _abiBinJson = require('./OrderEvent1.json');
var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));

contractName = Object.keys(_abiBinJson.contracts);
console.log("- contract name : ", contractName);
_abiArray=JSON.parse(_abiBinJson.contracts[contractName].abi); 
_bin = _abiBinJson.contracts[contractName].bin;

async function deploy() {
    const accounts = await web3.eth.getAccounts();
    console.log("Deploying the contract form " + accounts[0]);
    var deployed = await new web3.eth.Contract(_abiArray)
        .deploy({data: "0x" + _bin})
        .send({from: accounts[0], gas: 260000, function(err, transactionHash) {
            if(!err) {
                console.log("hash : " + transactionHash);
            }
        }})
    console.log("---> The contract deployed to : " + deployed.options.address);
}

deploy();

Overwriting src/OrderEvent1Deploy.js


In [15]:
!node src/OrderEvent1Deploy.js

- contract name :  [ 'src/OrderEvent1.sol:OrderEvent1' ]
Deploying the contract form 0xf0B05982bB5D9E5e85b2382075B18662A2570e16
---> The contract deployed to : 0xd05c691a2bD5F440226D2f6044CfBB0Da7c1AA0f


## 주문

In [16]:
%%writefile src/OrderUse.js
var Web3 = require('web3');
var fs = require('fs');
var _abiBinJson = require('./OrderEvent1.json');
var web3 = new Web3(new Web3.providers.WebsocketProvider("ws://localhost:8345"));

contractName = Object.keys(_abiBinJson.contracts);
console.log("- contract name : ", contractName);
_abiArray=JSON.parse(_abiBinJson.contracts[contractName].abi); 
_bin = _abiBinJson.contracts[contractName].bin;

async function doIt() {
    var _order = new web3.eth.Contract(_abiArray, '0xd05c691a2bD5F440226D2f6044CfBB0Da7c1AA0f');
    const accounts = await web3.eth.getAccounts();
    console.log("Account : " + accounts[0]);
    
    var event = _order.events.OrderLog(
        {fromBlock: 0}, function(error, result) {
            if(!error) {
                var log = JSON.stringify(result.returnValues);
                console.log("Event fired : " + log);
                fs.appendFile("src/OrderEvent.txt", log, "utf-8", function(e) {
                    if(!e) {
                        console.log(">> Writing to file");
                    }
                });
            }
            else {
                console.log(error);
            }
        }
    );
    
    var value;
    my = await _order.methods.order("0x1111", 3, 100, "20 2-gil Hongji-dong Jongro-gu Seoul")
            .send({from: accounts[0], gas: 100000, value: 300});
    console.log("---> MyFunction called " + JSON.stringify(my.events.OrderLog.returnValues));
    
    my = await _order.methods.order("0x1111", 5, 100, "20 2-gil Hongji-dong Jongro-gu Seoul")
            .send({from: accounts[0], gas: 100000, value: 500});
    console.log("---> MyFunction called " + JSON.stringify(my.events.OrderLog.returnValues));
    
    my = await _order.methods.order("0x1111", 20, 100, "20 2-gil Hongji-dong Jongro-gu Seoul")
            .send({from: accounts[0], gas: 100000, value: 2000});
    console.log("---> MyFunction called " + JSON.stringify(my.events.OrderLog.returnValues));
    
    process.exit(1);
}

doIt();

Writing src/OrderUse.js


In [17]:
!node src/OrderUse.js

- contract name :  [ 'src/OrderEvent1.sol:OrderEvent1' ]
Account : 0xf0B05982bB5D9E5e85b2382075B18662A2570e16
Event fired : {"0":"0xf0B05982bB5D9E5e85b2382075B18662A2570e16","1":"0x1111","2":"3","3":"100","4":"20 2-gil Hongji-dong Jongro-gu Seoul","_from":"0xf0B05982bB5D9E5e85b2382075B18662A2570e16","_itemId":"0x1111","_quantity":"3","unitPrice":"100","addr":"20 2-gil Hongji-dong Jongro-gu Seoul"}
---> MyFunction called undefined
>> Writing to file
Event fired : {"0":"0xf0B05982bB5D9E5e85b2382075B18662A2570e16","1":"0x1111","2":"5","3":"100","4":"20 2-gil Hongji-dong Jongro-gu Seoul","_from":"0xf0B05982bB5D9E5e85b2382075B18662A2570e16","_itemId":"0x1111","_quantity":"5","unitPrice":"100","addr":"20 2-gil Hongji-dong Jongro-gu Seoul"}
---> MyFunction called undefined
>> Writing to file
Event fired : {"0":"0xf0B05982bB5D9E5e85b2382075B18662A2570e16","1":"0x1111","2":"20","3":"100","4":"20 2-gil Hongji-dong Jongro-gu Seoul","_from":"0xf0B05982bB5D9E5e85b2382075B18662A2570e16","_itemId":"0

## 저장내역 출력

In [18]:
!type src\OrderEvent.txt

{"0":"0xf0B05982bB5D9E5e85b2382075B18662A2570e16","1":"0x1111","2":"3","3":"100","4":"20 2-gil Hongji-dong Jongro-gu Seoul","_from":"0xf0B05982bB5D9E5e85b2382075B18662A2570e16","_itemId":"0x1111","_quantity":"3","unitPrice":"100","addr":"20 2-gil Hongji-dong Jongro-gu Seoul"}{"0":"0xf0B05982bB5D9E5e85b2382075B18662A2570e16","1":"0x1111","2":"5","3":"100","4":"20 2-gil Hongji-dong Jongro-gu Seoul","_from":"0xf0B05982bB5D9E5e85b2382075B18662A2570e16","_itemId":"0x1111","_quantity":"5","unitPrice":"100","addr":"20 2-gil Hongji-dong Jongro-gu Seoul"}{"0":"0xf0B05982bB5D9E5e85b2382075B18662A2570e16","1":"0x1111","2":"20","3":"100","4":"20 2-gil Hongji-dong Jongro-gu Seoul","_from":"0xf0B05982bB5D9E5e85b2382075B18662A2570e16","_itemId":"0x1111","_quantity":"20","unitPrice":"100","addr":"20 2-gil Hongji-dong Jongro-gu Seoul"}
